In [1]:
from PIL import Image
from pytesseract import pytesseract

In [8]:
# Defining paths to tesseract.exe
# and the image we would be using
# path_to_tesseract = r"/usr/local/Cellar/tesseract/4.1.3/bin/tesseract"
path_to_tesseract='/home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/tesseract'
image_path = r"../MS-Online-Menu-Mains-January.jpg"

## Opening the image & storing it in an image object
img = Image.open(image_path)
  
# Providing the tesseract executable
# location to pytesseract library
pytesseract.tesseract_cmd = path_to_tesseract
  
# Passing the image object to image_to_string() function
# This function will extract the text from the image
text = pytesseract.image_to_string(img)
  
# Displaying the extracted text
print(text[:-1])

PermissionError: [Errno 13] Permission denied: '/home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/tesseract'

In [3]:
import pandas as pd
import numpy as np

In [3]:
text="""LONDON
HOUSE

A LA CARTE MENU

Marinated Nocellara olives 5
Harissa spiced nuts 3.5

Bread basket, olive oil & aged balsamic 4.5

STARTERS

Burrata, heirloom tomato, basil & toasted sourdough 11.5
Chicken Caesar salad, anchovies, croutons 1217.5
Hiramasa kingfish crudo, avocado, chili & ginger dressing 13.5

Chicken & duck liver parfait, brioche & grape chutney L6

MAINS
Harissa roasted cauliflower, warm chickpea salad, chimichurri 20.5
Grilled hake, romesco sauce, fennel & piquillo pepper 24
Parsley & basil chicken Kiev, creamed potatoes, salsa verde 18
Steak frites, roasted beef tomato, watercress 27
BBQ lamb, labneh, fennel and chili slaw 26.5

Fish & chips, tartare sauce, curry sauce, mushy peas 19.5

SIDES

Koffmann's fries 5.5

Heirloom tomato & basil salad 5.5

Seasonal greens & sage dressing 5.5

SWEETS
Chocolate brownic, dulce de leche, caramel sundae 9
Cinnamon doughnuts, passion [ruit curd 7
Sticky toffee pudding & vanilla ice cream 9

Mango sorbet, marinated berries 7.5

 

If you have a food allergy, intolerance or sensitivity, please speak lo your sereer

about ingredients in our dishes before you order your meal.

   

An optional 15% harge will be added to your All prices are inclusive of VAT."""

   


In [66]:
string_list=[x for x in text.split("\n") if len(x)!=0]

In [145]:
ingredient_file_path='../Emission_computing/final_ingredients_emissions.csv'
df=pd.read_csv(ingredient_file_path,error_bad_lines=False)

/home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 280: expected 2 fields, saw 4\n'


In [146]:
df[df['ingredient']=='ramen noodles']

,ingredient,emissions
513,ramen noodles,1374.0


In [161]:
string_list

['LONDON',
 'HOUSE',
 'A LA CARTE MENU',
 'Marinated Nocellara olives 5',
 'Harissa spiced nuts 3.5',
 'Bread basket, olive oil & aged balsamic 4.5',
 'STARTERS',
 'Burrata, heirloom tomato, basil & toasted sourdough 11.5',
 'Chicken Caesar salad, anchovies, croutons 1217.5',
 'Hiramasa kingfish crudo, avocado, chili & ginger dressing 13.5',
 'Chicken & duck liver parfait, brioche & grape chutney L6',
 'MAINS',
 'Harissa roasted cauliflower, warm chickpea salad, chimichurri 20.5',
 'Grilled hake, romesco sauce, fennel & piquillo pepper 24',
 'Parsley & basil chicken Kiev, creamed potatoes, salsa verde 18',
 'Steak frites, roasted beef tomato, watercress 27',
 'BBQ lamb, labneh, fennel and chili slaw 26.5',
 'Fish & chips, tartare sauce, curry sauce, mushy peas 19.5',
 'SIDES',
 "Koffmann's fries 5.5",
 'Heirloom tomato & basil salad 5.5',
 'Seasonal greens & sage dressing 5.5',
 'SWEETS',
 'Chocolate brownic, dulce de leche, caramel sundae 9',
 'Cinnamon doughnuts, passion [ruit curd 7

In [211]:
final_dict={'ingredient_parsed':[],"ingredient_from_df":[], 'emission':[]}

for line in string_list:
    line=re.sub('[^A-Za-z0-9]+', ' ', line)
    line=re.sub(r'[0-9]+', '', line)
    ingredient_words=re.split('\s+', line)
    for ingredient in ingredient_words:
        if df[df['ingredient'].str.match(r''+str(ingredient)+'$')==True].ingredient.values.size>0:
            final_dict['ingredient_from_df'].append(df[df['ingredient'].str.match(r''+str(ingredient)+'$')== True].ingredient.iloc[0])
            final_dict['emission'].append(df[df['ingredient'].str.match(r''+str(ingredient)+'$')== True].emissions.iloc[0])
            final_dict["ingredient_parsed"].append(line)

In [212]:
final_df=pd.DataFrame(final_dict)

In [213]:
final_df['ingredient_parsed']=final_df['ingredient_parsed'].astype(str)

In [214]:
final_df.groupby('ingredient_parsed').sum()

,emission
ingredient_parsed,
BBQ lamb labneh fennel and chili slaw,22844.0
Burrata heirloom tomato basil toasted sourdough,9807.0
Chicken duck liver parfait brioche grape chutney L,6710.0
Cinnamon doughnuts passion ruit curd,2445.0
Fish chips tartare sauce curry sauce mushy peas,4250.0
Grilled hake romesco sauce fennel piquillo pepper,6381.0
Harissa roasted cauliflower warm chickpea salad chimichurri,971.0
Heirloom tomato basil salad,3228.0
Hiramasa kingfish crudo avocado chili ginger dressing,2471.0
